# Performance Measurement and Improvement Techniques

## Goal
Image processing에서는 초당 많은 작업을 처리하기 때문에 올바른 솔루션을 제공하는 것뿐만 아니라 빠르게 동작하는 코드를 작성해야 한다.

따라서 여기서는 다음의 내용을 배우게 된다:
- 성능 측정 방법
- 성능을 향상시키기 위한 몇가지 팁
- 함수의 사용법 : `cv2.getTickCount`, `cv2.getTickFrequency`, 등

OpenCV외에 파이썬이 제공하는 `time` 모듈도 실행 시간을 측정하는데 도움이 된다. 또한 `profile` 모듈도 함수 실행에 걸린 시간, 함수 호출 수 등의 자세한 report를 얻는데 도움을 준다.

IPython을 사용한다면, 이러한 기능들은 사용자 친화적으로 통합되어 있다. 여기서는 몇가지 중요한 내용들을 다루며 자세한 내용은 마지막의 Additional Resources를 참고하면 된다.

### Measuring Performance with OpenCV

`cv2.getTickCount` 함수는 참조 이벤트(장치의 전원이 켜진 순간과 같은) 이후 함수 실행 까지의 clock-cycle의 수를 반환한다. 따라서, 이를 함수 실행 전과 후에 호출하면 함수 실행에 사용된 clock-cycle을 얻을 수 있다.

In [23]:
import cv2
import numpy as np

def execution():
    print('Function executed')

    
e1 = cv2.getTickCount()
execution()
e2 = cv2.getTickCount()

t = (e2-e1)/cv2.getTickFrequency()
print('clock-cycle :', t)

Function executed
clock-cycle : 0.000667258


`cv2.getTickFrequency` 함수는 clock-cycle의 주파수 또는 초당 clock-cycle 수를 반환한다. 따라서 clock-cycle수를 이 값으로 나눠주면 실행 시간을 초단위로 얻을 수 있다.

다음은 5에서 49까지의 홀수 크기 커널로 median filtering을 적용하는 동안 실행 시간을 초 단위로 얻는 예제이다.

In [2]:
img = cv2.imread('./images/messi5.jpg')
e1 = cv2.getTickCount()
for i in range(5, 49, 2):
    img = cv2.medianBlur(img, i)

e2 = cv2.getTickCount()
t = (e2 - e1) / cv2.getTickFrequency()

print('elaped time [sec] :', t)

elaped time [sec] : 0.294836531


#### Note
`cv2.getTickCount()` 대신에 time 모듈의 `time.time()`을 사용해서 위와 같은 작업을 수행할 수 있다.

### Default Optimization in OpenCV

많은 OpenCV 함수들은 SSE2, AVX 등을 통해 최적화 되어있다.(최적화되지 않은 코드들도 포함되어 있음) 따라서 만약 시스템이 이러한 기능들을 지원한다면(거의 모든 최신 프로세서들은 이들을 지원함) 우리는 이들을 활용해야 한다.

이러한 최적화 기능들은 컴파일 시 기본으로 활성화가 되어 있으며 OpenCV는 활성화가 되어 있다면 최적화된 코드를 실행하고, 아니라면 최적화되지 않은 코드를 실행하게 된다. 

활성화/비활성화의 여부는 `cv2.useOptimized()`를 통해 확인할 수 있고, `cv2.setUseOptimized()`를 통해 활성화/비활성화를 설정할 수 있다.

다음은 활성화 시의 성능과 비활성화 시의 성능을 비교하는 예제이다.

In [3]:
# 최적화 활성화 여부 검사
# 기본으로 활성화 되어 있음
print(cv2.useOptimized())

True


In [4]:
# 최적화 활성화 시 성능 출력
%timeit res = cv2.medianBlur(img, 49)

10.4 ms ± 57.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
# 최적화 비활성화
cv2.setUseOptimized(False)
print(cv2.useOptimized())

False


In [6]:
# 최적화 비활성화 시 성능 출력
%timeit res = cv2.medianBlur(img, 49)

10.8 ms ± 62.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


위의 결과를 통해 최적화를 한 경우가 최적화를 하지 않은 경우보다 빠른 것을 확인할 수 있다. median filtering의 소스코드를 확인해보면 SIMD 최적화가 되어있는 것을 확인할 수 있다. 따라서 이를 사용하기 위해서는 최적화를 활성화 하여야 한다.(기본으로 활성화 되어 있다는 것을 기억하기)

### Mesuring Performance in IPython

때때로 비슷한 두가지 연산의 성능을 비교하여야 하는 경우가 있다. IPython에서는 magic command인 timeit을 통해 이를 수행할 수 있다.

timeit은 코드를 정확한 결과를 얻기 위해 여러 번 실행한다. 따라서 이는 single line의 코드를 측정하기에 적합하다.

timeit을 통해 다음과 같은 여러 형태의 덧셈 연산의 성능을 측정해보자.
```
x = 5; y = x**2
x = 5; y = x*x
x = np.uint8([5]); y = x*x
x = np.uint8([5]); y = np.square(x)
```

In [7]:
x = 5
%timeit y = x**2

157 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [8]:
x = 5
%timeit y = x*x

41.2 ns ± 0.499 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [9]:
x = np.uint8([5])
%timeit y = x*x

368 ns ± 5.18 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
x = np.uint8([5])
%timeit y = np.square(x)

368 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


위의 결과를 통해 `x = 5; y = x*x`가 Numpy 연산에 비해 약 10배 정도 빠른 것을 확인할 수 있다. 만약 행렬을 생성한다면, 이는 100배 이상 빨라질 수 있을 것이다.

#### Note

파이썬의 스칼라 연산은 넘파이의 스칼라 연산보다 훨씬 빠르다. 따라서 원소 수가 적은 연산의 경우 파이썬 스칼라 연산을 사용하는 것이 좋다. 넘파이는 행렬의 크기가 클 경우에 좋은 결과를 얻을 수 있다.

이번에는 `cv2.countNonZero()`와 `np.count_nonzero()`를 비교해보자.

In [25]:
%timeit z = cv2.countNonZero(img[:, :, 0])

64.8 µs ± 317 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
%timeit z = np.count_nonzero(img[:, :, 0])

333 µs ± 2.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


OpenCV 함수가 약 5배 빠른 것을 확인할 수 있다.

#### Note
일반적으로, OpenCV 함수는 Numpy 함수보다 빠르다. 따라서 같은 연산이라면, OpenCV 함수의 사용을 권장한다.

하지만 몇가지 예외의 경우는 있다. 특히 Numpy에서 copy 대신 view로 동작하는 경우이다.

### More IPython magic commands

여기서 다루지 않은 성능 측정, profiling, line profiling, 메모리 측정 등의 몇가지 magic command들이 더 있다. 이들은 모두 잘 문서화가 되어있으며 필요한 경우 찾아서 활용할 수 있다.

### Performance Optimization Techniques
파이썬과 넘파이의 최대 성능 활용을 위한 몇가지 기술 및 코딩 방법이 있다. 여기서는 관련된 몇가지만 언급하고 추가로 링크를 제공한다. 

여기서 주목해야 할 것은 먼저, 간단한 방식으로 알고리즘을 구현하려고 시도하는 것이다. 일단 동작하면, profile하고, 병목을 찾은 후 최적화 하는 것이다.

1. 파이썬에서 루프의 사용은 최대한 피해야 한다. 특히 double/triple 루프 등이다. 그들은 본질적으로 느리다.
2. 알고리즘/코드를 가능한 최대로 벡터화 한다. 왜냐하면 Numpy와 OpenCV는 벡터 연산에 최적화 되어 있기 때문이다.
3. 캐시 일관성(cache coherence)를 활용한다.
4. 필요한 경우가 아니라면 행렬의 복사는 절대 하지 않는다. view를 대신 사용하도록 해야 한다. 행렬의 복사는 costly operation이다.

만약, 이러한 모든 연산을 사용해도 느리거나 또는 거대한 루프를 피할 수 없다면, 가속화를 위해 Cython과 같은 추가적인 라이브러리를 사용해야 한다.

### Additional Resources

1. [Python Optimization Techniques](http://wiki.python.org/moin/PythonSpeed/PerformanceTips)
2. Scipy Lecture Notes - [Advanced Numpy](http://scipy-lectures.github.io/advanced/advanced_numpy/index.html#advanced-numpy)
3. [Timing and Profiling in IPython](http://pynash.org/2013/03/06/timing-and-profiling/)